In [1]:
!pip install nltk

In [3]:
import torch
import numpy as np
import nltk
from nltk.stem.porter import PorterStemmer

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [12]:
!python nlp_utils.py
!python model.py
!python train.py

Epoch [100/800], Loss: 1.0008
Epoch [200/800], Loss: 0.2608
Epoch [300/800], Loss: 0.0136
Epoch [400/800], Loss: 0.0056
Epoch [500/800], Loss: 0.0046
Epoch [600/800], Loss: 0.0014
Epoch [700/800], Loss: 0.0024
Epoch [800/800], Loss: 0.0009
final loss: 0.0009
training complete. file saved to data.pth


In [16]:
#main

import random
import json
import torch

from model import NeuralNet
from nlp_utils import tokenize, bag_of_words

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('intents.json', 'r') as f:
  intents = json.load(f)

FILE = 'data.pth'
data = torch.load(FILE)

input_size = data['input_size']
output_size = data['output_size']
hidden_size = data['hidden_size']
all_words = data['all_words']
tags = data['tags']
model_state = data['model_state']

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

bot_name = 'Almost Human'
print("Let's chat! (type 'quit' to exit)")
while True:
  sentence = input('You: ')

  if(sentence == 'quit'):
    break
  
  sentence = tokenize(sentence)
  X = bag_of_words(sentence, all_words)
  X = X.reshape(1, X.shape[0])
  X = torch.from_numpy(X)

  output = model(X)
  _, predicted = torch.max(output, dim=1)
  tag = tags[predicted.item()]

  probs = torch.softmax(output, dim=1)
  prob = probs[0][predicted.item()]

  if(prob > 0.75):
    for intent in intents['intents']:
      if(tag == intent['tag']):
        print(f"{bot_name}: {random.choice(intent['responses'])}")
  else:
    print(f"{bot_name}: Sorry, I don't understand....")



Let's chat! (type 'quit' to exit)
You: Hi
Almost Human: Hi there, how can I help?
You: Hi
Almost Human: Hi there, what can I do for you?
You: What do you sell ?
Almost Human: We have coffee and tea
You: How long takes shipping ?
Almost Human: Delivery takes 2-4 days
You: Can you help me ?
Almost Human: Sorry, I don't understand....
You: Crack a joke
Almost Human: What did the buffalo say when his son left for college? Bison.
You: Bye
Almost Human: Have a nice day
You: quit
